In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/industrial-ai-examen/data/data_pdmp6.csv
/kaggle/input/industrial-ai-examen/data/data_po3.csv
/kaggle/input/industrial-ai-examen/data/data_pin2.csv
/kaggle/input/industrial-ai-examen/data/data_pin3.csv
/kaggle/input/industrial-ai-examen/data/data_pin5.csv
/kaggle/input/industrial-ai-examen/data/data_pin1.csv
/kaggle/input/industrial-ai-examen/data/data_po2.csv
/kaggle/input/industrial-ai-examen/data/data_po5.csv
/kaggle/input/industrial-ai-examen/data/data_pdmp2.csv
/kaggle/input/industrial-ai-examen/data/data_pin4.csv
/kaggle/input/industrial-ai-examen/data/data_po4.csv
/kaggle/input/industrial-ai-examen/data/data_po6.csv
/kaggle/input/industrial-ai-examen/data/data_po1.csv
/kaggle/input/industrial-ai-examen/data/data_pdmp4.csv
/kaggle/input/industrial-ai-examen/data/data_pdmp1.csv
/kaggle/input/industrial-ai-examen/data/data_pin6.csv
/kaggle/input/industrial-ai-examen/data/data_pdmp5.csv
/kaggle/input/industrial-ai-examen/data/data_pdmp3.csv


In [2]:


test_1 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp3.csv',header=None)
test_2 =  pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin3.csv',header=None)
test_3 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po3.csv',header=None)

#Target feature
df_target= test_1.iloc[:,0]
test_1 = test_1.iloc[:,1:]
test_2 = test_2.iloc[:,1:]
test_3 = test_3.iloc[:,1:]


# Concat the differntial pressure dataset
drill_1 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp1.csv',header=None)
drill_2 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp2.csv', header=None)
drill_4 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp4.csv',header=None)
drill_5 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp5.csv', header=None)
drill_6 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pdmp6.csv',header=None)
drill_pd = pd.concat([drill_1,drill_2,drill_4,drill_5,drill_6], ignore_index=True)

#Concat the pin dataset

drill_pin1 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin1.csv',header=None)
drill_pin2 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin2.csv',header=None)
drill_pin4 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin4.csv',header=None)
drill_pin5 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin5.csv',header=None)
drill_pin6 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_pin6.csv',header=None)
drill_pin = pd.concat([drill_pin1,drill_pin2,drill_pin4,drill_pin5,drill_pin6], ignore_index=True)

# Concat the po dataset

drill_po1 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po1.csv',header=None)
drill_po2 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po2.csv',header=None)
drill_po4 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po4.csv',header=None)
drill_po5 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po5.csv',header=None)
drill_po6 = pd.read_csv('/kaggle/input/industrial-ai-examen/data/data_po6.csv',header=None)
drill_po = pd.concat([drill_po1,drill_po2,drill_po4,drill_po5,drill_po6], ignore_index=True)

Target= drill_pd.iloc[:,0]
pd_data = drill_pd.iloc[:,1:]
pin_data = drill_pin.iloc[:,1:]
po_data = drill_po.iloc[:,1:]

target = pd.DataFrame(Target)
target.columns = ['Fault']

In [3]:



def pressure_change(dataset):
  """
  Calculates pairwise differences between consecutive columns for each row in a DataFrame.

  Args:
      dataset (pandas.DataFrame): The input DataFrame containing data.

  Returns:
      pandas.DataFrame: A new DataFrame with a column named 'difference' containing 
          the pairwise differences for each row.

  Raises:
      ValueError: If the input is not a pandas DataFrame.
  """

  if not isinstance(dataset, pd.DataFrame):
    raise ValueError("Input must be a pandas DataFrame.")

  result_df = pd.DataFrame(columns=['row_index', 'difference'])

  for row_index in dataset.index:
    row_data = dataset.iloc[row_index]
    differences = []
    for i in range(1, len(row_data)):
      differences.append(row_data.iloc[i] - row_data.iloc[i - 1])
    result_df.loc[len(result_df)] = [row_index, differences]

  return result_df





In [4]:
# Pressure changes capturing
x = pressure_change(pd_data)
y1 = pressure_change(pin_data)
z = pressure_change(po_data)
x1 = pressure_change(test_1)
x2 = pressure_change(test_2)
x3 = pressure_change(test_3)

In [5]:
#test data making
df1 =pd.DataFrame(columns=['pd_min','pd_max','pin_min','pin_max','po_min','po_max'])
df1['pd_min'] = x1['difference'].apply(min)
df1['pd_max'] = x1['difference'].apply(max)
df1['pin_max'] = x2['difference'].apply(max)
df1['pin_min'] = x2['difference'].apply(min)
df1['po_max'] = x3['difference'].apply(max)
df1['po_min'] = x3['difference'].apply(min)

#Training data making

df =pd.DataFrame(columns=['pd_min','pd_max','pin_min','pin_max','po_min','po_max'])
df['pd_min'] = x['difference'].apply(min)
df['pd_max'] = x['difference'].apply(max)
df['pin_max'] = y1['difference'].apply(max)
df['pin_min'] = y1['difference'].apply(min)
df['po_max'] = z['difference'].apply(max)
df['po_min'] = z['difference'].apply(min)


**Decision tree Classifier**

In [6]:

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


X = df
y = target['Fault']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Accuracy: 0.765909535931075
Confusion Matrix:
 [[ 508    0   54    6  184   35    9    7    2   92    0]
 [   0 1010    0   12    0    0    4    8    1    2    8]
 [  47    0  660    1   82   27    7   17   16   87    0]
 [   9    9    2  873    9    0    0    3    6    7    0]
 [ 184    0   60   11  513   26   19   21    4  104    1]
 [  27    0   36    0   24  770    9   17    4   32    0]
 [   6    2    9    1   12   11  655  216    0   21    7]
 [  16    6   19    4   22   18  191  579    3   27    1]
 [   0    2   17    9    5    3    5    8  857    8    0]
 [  69    3   87    5  105   48   28   28   14  458    2]
 [   0    9    0    1    0    0    9    1    1    0  940]]
Classification Report:
               precision    recall  f1-score   support

           1       0.59      0.57      0.58       897
           2       0.97      0.97      0.97      1045
           3       0.70      0.70      0.70       944
           4       0.95      0.95      0.95       918
           5       

**Without hyper tunning Score .10**

In [7]:
drill3_pred = dt.predict(df1)
drill3_pred = pd.DataFrame(drill3_pred,columns=['Target'])
drill3_pred['ID'] = drill3_pred.index
drill3_pred = drill3_pred[['ID','Target']]
drill3_pred.to_csv('submission 6.csv')

**Results with hypertunning**

In [8]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best Hyperparameters:", best_params)

# Use the best hyperparameters to initialize the Decision Tree classifier
dt_best = DecisionTreeClassifier(**best_params, random_state=42)

# Train the model with the best hyperparameters
dt_best.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt_best.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}
Accuracy: 0.7806931662424124
Confusion Matrix:
 [[ 531    0   27    1  206   23    6    6    1   96    0]
 [   0 1003    0   14    1    0    3   13    1    2    8]
 [  55    0  641    2  100   21    5   15   13   92    0]
 [   9    9    2  870   13    1    0    1    6    7    0]
 [ 166    0   40    1  601   18   17   14    4   82    0]
 [  32    0   29    0   17  773    8   18    4   38    0]
 [   6    3    5    0   24   11  640  231    0   13    7]
 [  15    6    7    2   28   10  149  629    3   36    1]
 [   2    1   16    9    5    3    1   10  856    9    2]
 [  78    2   59    3  114   47   19   28   11  486    0]
 [   0    6    0    1    0    0    7    2    1    0  944]]
Classification Report:
               precision    recall  f1-score   support

           1       0.59      0.59      0.59       897
          

**with hypertunning score**

In [9]:
drill3_pred_tune = dt_best.predict(df1)
drill3_pred_tune = pd.DataFrame(drill3_pred,columns=['Target'])
drill3_pred_tune['ID'] = drill3_pred.index
drill3_pred_tune = drill3_pred[['ID','Target']]
drill3_pred_tune.to_csv('submission 7.csv')